### RECOMMENDERS

Here we provide the following recommenders:
   - Popularity recommender: the best rated recipes.
   - Collaborative filtering recommender: uses item-item similarities based on users in common, using nearest neighbors searching method for training.
   - Content-based recommender: the similarity between the items recommended is determined by the content of those items rather than learned from user interaction data.

In [1]:
import numpy as np
import pandas as pd
import csv
import graphlab as gl

In [2]:
df = pd.read_csv("ratings.csv")

In [3]:
df.head(10)

,user,recipe,rating
0,user_1,236853,4
1,user_1,241469,5
2,user_1,85980,5
3,user_2,224046,5
4,user_2,297457,5
5,user_2,299052,3
6,user_2,297612,3
7,user_2,297836,4
8,user_2,224507,5
9,user_3,227885,4


In [4]:
sf = gl.SFrame(data=df)

This non-commercial license of GraphLab Create for academic use is assigned to gonzaloespinosaduelo@gmail.com and will expire on November 08, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1511820402.log


In [5]:
sf.shape

(59742, 3)

In [16]:
recipe_rating_stats = sf.groupby(key_columns='recipe',
                                operations={'mean_rating': gl.aggregate.MEAN('rating'), 
                                            'std_rating': gl.aggregate.STD('rating')})
recipe_rating_stats

recipe,mean_rating,std_rating
broken-biscuit-cakes,2.0,1.0
299026,3.14285714286,0.989743318611
169960,3.5,0.866025403784
298082,4.66666666667,0.471404520791
299894,4.57142857143,0.728431359085
206270,2.6,1.2
204980,4.25,0.433012701892
94097,4.0,0.0
chicken-cherry-salad,4.4,0.8
299180,1.0,0.0


In [17]:
# top rated
user_rating_stats = sf.groupby(key_columns='user',
                                operations={'mean_rating': gl.aggregate.MEAN('rating'), 
                                            'std_rating': gl.aggregate.STD('rating')})
user_rating_stats

user,mean_rating,std_rating
user_1475,3.5,1.58113883008
user_960,3.0,1.58113883008
user_7183,3.66666666667,0.942809041582
user_2594,3.66666666667,0.942809041582
user_9554,3.16666666667,1.06718737291
user_8898,4.5,0.763762615826
user_9654,3.16666666667,1.34370962472
user_6537,4.33333333333,0.666666666667
user_6414,3.88888888889,0.99380799
user_9930,3.0,1.0


### Popularity recommender

In [6]:
popularity_model = gl.popularity_recommender.create(sf, user_id='user', item_id='recipe', target='rating')

Recsys training: model = popularity

Preparing data set.

Data has 59742 observations with 10000 users and 15210 items.

Data prepared in: 0.13238s

59742 observations to process; with 15210 unique items.

In [10]:
# get the top for a user
popularity_recomm = popularity_model.recommend(users=["user_2"], k=5)
popularity_recomm.print_rows(num_rows=25)

+--------+--------+-------+------+
|  user  | recipe | score | rank |
+--------+--------+-------+------+
| user_2 | 296621 |  5.0  |  1   |
| user_2 | 80843  |  5.0  |  2   |
| user_2 | 268707 |  5.0  |  3   |
| user_2 | 300037 |  5.0  |  4   |
| user_2 | 241469 |  5.0  |  5   |
+--------+--------+-------+------+
[5 rows x 4 columns]



In [18]:
popularity_recomm.save("popularity_recommendations.csv")

### Item-Item recommender
Create a recommender that uses item-item similarities based on users in common.

#### Using Pearson correlation

In [22]:
itemitem_pearson = gl.item_similarity_recommender.create(sf, user_id='user', item_id='recipe', target='rating', 
                                                     similarity_type='pearson')
pred_itemitem_pearson = itemitem_pearson.predict(sf)
recs_itemitem_pearson = itemitem_pearson.recommend()     # top 10 collaborative filtering recomendations
recs_itemitem_pearson.save("itemitem_pearson_recommendations.csv")

Recsys training: model = item_similarity

Preparing data set.

Data has 59742 observations with 10000 users and 15210 items.

Data prepared in: 0.127193s

Training model from provided data.

...

recommendations finished on 6000/10000 queries. users per second: 13444

recommendations finished on 7000/10000 queries. users per second: 12836

recommendations finished on 8000/10000 queries. users per second: 12695.5

recommendations finished on 9000/10000 queries. users per second: 12810.3

recommendations finished on 10000/10000 queries. users per second: 12457.2

In [24]:
recs_itemitem_pearson

user,recipe,score,rank
user_1,96181,5.0,1
user_1,205980,5.0,2
user_1,295802,5.0,3
user_1,300909,5.0,4
user_1,sticky-toffee-puddings,5.0,5
user_1,296256,5.0,6
user_1,296621,5.0,7
user_1,80843,5.0,8
user_1,268707,5.0,9
user_1,300037,5.0,10


#### Using cosine correlation

In [19]:
itemitem_cos = gl.item_similarity_recommender.create(sf, user_id='user', item_id='recipe', target='rating', 
                                                     similarity_type='cosine')
pred_itemitem_cos = itemitem_cos.predict(sf)
recs_itemitem_cos = itemitem_cos.recommend()     # top 10 collaborative filtering recomendations

Recsys training: model = item_similarity

Preparing data set.

Data has 59742 observations with 10000 users and 15210 items.

Data prepared in: 0.155246s

Training model from provided data.

...

recommendations finished on 6000/10000 queries. users per second: 11402.4

recommendations finished on 7000/10000 queries. users per second: 11364.2

recommendations finished on 8000/10000 queries. users per second: 11542.1

recommendations finished on 9000/10000 queries. users per second: 11699.8

recommendations finished on 10000/10000 queries. users per second: 11224.7

In [20]:
itemitem_cos.recommend(users=["user_1"], k=10)

user,recipe,score,rank
user_1,tuscan-basil-and-garlic-panzanella ...,0.0,1
user_1,93480,0.0,2
user_1,13403,0.0,3
user_1,227885,0.0,4
user_1,224507,0.0,5
user_1,297836,0.0,6
user_1,297612,0.0,7
user_1,299052,0.0,8
user_1,297457,0.0,9
user_1,224046,0.0,10


In [49]:
recs_itemitem_cos.save("itemitem_cos_recommendations.csv")